# Are-MipiA1A2 : Venin

In [13]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


class agent :
    def __init__(self):
        self.efficacite= 0.1
        self.spawn = 0.1
        self.coagulation = 0.1
        self.perte = 0.1

globule_rouge = agent()
globule_blanc = agent()
plaquette = agent()
venin = agent()
coagulation = agent()
anti_coagulant = agent()



"""Type : Cell = List[float**6]"""

'Type : Cell = List[float**6]'

In [32]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,0)
    return matrix
            

In [15]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(n):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [83]:
def selfinteract(mat,N):
    matrix = np.zeros((N,N,7))
    for i in range(N):
        for j in range(N):
            """
            plaquettes + plaquettes => -plaquettes +coagulation
            globules blanc + venin =>  - globules blanc -venin
            globules rouges + venin => -globules rouges -venin
            coagulation + anticoagulant => -coagulation -anticoagulant
            venin + anticoagulant => -venin -anticoagulant
            globules blanc + coagulation => globules blancs - coagulation
            
            interact(mat,i,j,i,j,"Pl","Pl")
            interact(mat,i,j,i,j,"Gb","Ve")
            interact(mat,i,j,i,j,"Gr","Ve")
            interact(mat,i,j,i,j,"Co","Ac")
            interact(mat,i,j,i,j,"Ve","Ac")
            interact(mat,i,j,i,j,"Gb","Co") """
            
            a=[]
            a = mat[i,j]
              
            #globule_rouge+venin
            #a[0] = a[0] - a[0]*venin.efficacite
            #globule_blanc+venin[]
            #a[3] = a[3] - a[1]*globule_blanc.efficacite
            #a[1] = 0
            #a[6] = 1
            #plaquettes+plaquettes
            #a[4] = a[4] + a[2]*plaquettes.coagulation
            #a[2] = a[2] - a[2]/plaquettes.coagulation
            #coagulation+anti_coagulant
            #a[4] = a[4] - a[5]*anti_coagulant.efficacite
            #a[5] = a[5] - a[5]/anti_coagulant.efficacite
            #globule_blanc+coagulation
            #a[4] = a[4] - a[1]*globule_blanc.efficacite
            #venin+anti_coagulant
            #a[3] = a[3] - a[5]*anti_coagulant.efficacite
            #a[5] = a[5] - a[5]/anti_coagulant.efficacite
            print(a)
            matrix[i,j] = a
    return matrix
            

In [84]:
mat=init_matrice(2)
print(mat)
selfinteract(mat,2)

[[[9.56144333e-01 1.00868033e-03 4.28469862e-02 0.00000000e+00
   4.28469862e-02 0.00000000e+00 0.00000000e+00]
  [9.29092990e-01 1.63086124e-03 6.92761491e-02 0.00000000e+00
   6.92761491e-02 0.00000000e+00 0.00000000e+00]]

 [[9.02154752e-01 2.25044070e-03 9.55948073e-02 0.00000000e+00
   9.55948073e-02 0.00000000e+00 0.00000000e+00]
  [9.76756923e-01 5.34590775e-04 2.27084864e-02 0.00000000e+00
   2.27084864e-02 0.00000000e+00 0.00000000e+00]]]
[0.95614433 0.00100868 0.04284699 0.         0.04284699 0.
 0.        ]
[0.92909299 0.00163086 0.06927615 0.         0.06927615 0.
 0.        ]
[0.90215475 0.00225044 0.09559481 0.         0.09559481 0.
 0.        ]
[9.76756923e-01 5.34590775e-04 2.27084864e-02 0.00000000e+00
 2.27084864e-02 0.00000000e+00 0.00000000e+00]


array([[[9.56144333e-01, 1.00868033e-03, 4.28469862e-02, 0.00000000e+00,
         4.28469862e-02, 0.00000000e+00, 0.00000000e+00],
        [9.29092990e-01, 1.63086124e-03, 6.92761491e-02, 0.00000000e+00,
         6.92761491e-02, 0.00000000e+00, 0.00000000e+00]],

       [[9.02154752e-01, 2.25044070e-03, 9.55948073e-02, 0.00000000e+00,
         9.55948073e-02, 0.00000000e+00, 0.00000000e+00],
        [9.76756923e-01, 5.34590775e-04, 2.27084864e-02, 0.00000000e+00,
         2.27084864e-02, 0.00000000e+00, 0.00000000e+00]]])

In [39]:
#def interact
def main(N,n):
    
    stop = False
    mat = init_matrice(N)
    rlist=[]
    
    while not stop:        
        
        mat = selfinteract(mat,N)
            
            
            
            
            
        """    
            
        I = randomlsit(N)
        J = randomlist(N)
        
        for i in I:
            for j in J:
                
                interact(mat,i,j)
        
        mat = spawn(mat,N)
        
        rlist.append(mat.copy)
        
        stop = mort(mat) """

        
    

In [20]:
"""def dectohex(n):
    p=8
    hexlist=['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    binlist=[0,0,0,0,0,0,0,0]
    r=''
    while p!=0:
        p=p-1
        if 2**p <= n:
            n=n-2**p
            binlist[p] = 1
    c=0
    s=0
    for i in range(0,4):
        s = s+ (2**i)*binlist[i+4]
        c=c+1
    r = '#'+ hexlist[s] 
    
    c=0
    s=0
    for i in range(0,4):
        s = s+ (2**i)*binlist[i]
        c=c+1
    r = r + hexlist[s] +"0000"
        
    print(binlist)
    return r """

'def dectohex(n):\n    p=8\n    hexlist=[\'0\',\'1\',\'2\',\'3\',\'4\',\'5\',\'6\',\'7\',\'8\',\'9\',\'A\',\'B\',\'C\',\'D\',\'E\',\'F\']\n    binlist=[0,0,0,0,0,0,0,0]\n    r=\'\'\n    while p!=0:\n        p=p-1\n        if 2**p <= n:\n            n=n-2**p\n            binlist[p] = 1\n    c=0\n    s=0\n    for i in range(0,4):\n        s = s+ (2**i)*binlist[i+4]\n        c=c+1\n    r = \'#\'+ hexlist[s] \n    \n    c=0\n    s=0\n    for i in range(0,4):\n        s = s+ (2**i)*binlist[i]\n        c=c+1\n    r = r + hexlist[s] +"0000"\n        \n    print(binlist)\n    return r '